In [37]:
from importlib import reload

In [1]:
import json
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.losses import sparse_categorical_crossentropy # used for integer targets
from tensorflow.keras.optimizers import Nadam
from random import shuffle, seed
from tqdm import tqdm, tqdm_notebook
import numpy as np
import os
import random

Using TensorFlow backend.


In [39]:
tf = reload(tf)

Finding all titles, training, and target sets composed by Chopin

In [4]:
titles = []
train = []
target = []
with open('data/Chopin_input.json', 'r') as handle:
    for i,line in enumerate(tqdm(handle)):
        song = json.loads(line)
        titles.append(song['title'])
        train.append(song['train'])
        target.append(song['target'])



0it [00:00, ?it/s]

1it [00:07,  7.28s/it]

2it [00:09,  5.66s/it]

3it [00:12,  4.86s/it]

4it [00:15,  4.28s/it]

5it [00:18,  3.86s/it]

6it [00:24,  4.64s/it]

7it [00:27,  4.24s/it]

8it [00:46,  8.44s/it]

9it [00:48,  6.78s/it]

10it [00:49,  5.02s/it]

11it [00:54,  4.86s/it]

12it [01:03,  6.26s/it]

13it [01:12,  6.95s/it]

14it [01:14,  5.60s/it]

15it [01:18,  4.98s/it]

16it [01:34,  8.22s/it]

17it [01:37,  6.63s/it]

18it [01:40,  5.68s/it]

19it [01:45,  5.36s/it]

20it [01:47,  4.50s/it]

21it [01:59,  6.64s/it]

22it [02:07,  7.24s/it]

23it [02:09,  5.66s/it]

24it [02:10,  4.17s/it]

25it [02:11,  3.13s/it]

26it [02:12,  2.42s/it]

27it [02:14,  2.39s/it]

28it [02:18,  2.79s/it]

29it [02:21,  2.98s/it]

30it [02:24,  2.84s/it]

31it [02:36,  5.86s/it]

32it [02:39,  4.74s/it]

33it [02:42,  4.36s/it]

34it [02:45,  3.90s/it]

35it [02:49,  4.08s/it]

36it [02:51,  3.23s/it]

37it [02:55,  3.61s/it]

38it [02:59,  3.62s/it]

39it [03:02,  3.59s/it]

40it [03:04,

Getting the unique Chopin songs and indices

In [6]:
unique_chopin = []
keep_indices = []
for i,song in enumerate(titles):
    if 'andante' in song:
        song = 'andante op 22'
    if 'ballade no. 2' in song:
        song = 'ballade no 2'
    if 'barcarolle' in song:
        song = 'barcarolle'
    if 'berceuse' in song:
        song = 'barceuse'
    if 'etude op. 10 no. 1' in song:
        song = 'etude op. 10 no. 1'
    if 'etude op. 10 no. 12' in song:
        song = 'etude op. 10 no. 12'
    if 'etude op. 10 no. 2' in song:
        song = 'etude op. 10 no. 2'
    if 'etude op. 10 no. 4' in song:
        song = 'etude op. 10 no. 4'
    if 'etude op. 10 no. 8' in song:
        song = 'etude op. 10 no. 8'
    if 'etude op. 25 no. 1' in song:
        song = 'etude op. 25 no. 1'
    if 'etude op. 25 no. 10' in song:
        song = 'etude op. 25 no. 10'
    if 'etude op. 25 no. 11' in song:
        song = 'etude op. 25 no. 11'
    if 'etude op. 25 no. 12' in song:
        song = 'etude op. 25 no. 12'
    if 'etude op. 25 no. 6' in song:
        song = 'etude op. 25 no. 6'
    if 'op. 49' in song:
        song = 'fantasy op. 49'
    if 'polonaise-fantasie' in song:
        song = 'polonaise-fantasie'
    if 'scherzo no. 2' in song:
        song = 'scherzo no. 2'
    if 'scherzo no. 3' in song:
        song = 'scherzo no. 3'
    if 'scherzo no. 4' in song:
        song = 'scherzo no. 4'
    if 'sonata no. 3' in song:
        song = 'sonata no. 3'
    if song not in unique_chopin:
        unique_chopin.append(song)
        keep_indices.append(i)

In [9]:
unique_train = []
for i,each in enumerate(train):
    if i in keep_indices:
        unique_train.append(each)

In [11]:
unique_target = []
for i, each in enumerate(target):
    if i in keep_indices:
        unique_target.append(each)

In [12]:
len(unique_train)

107

In [13]:
len(unique_target)

107

In [14]:
len(unique_chopin)

107

In [143]:
with open('data/chopin_titles.json', 'w') as to_write:
    for each in unique_chopin:
        json.dump(each, to_write)
        to_write.write('\n')

## Modeling

In [15]:
def create_model(seq_len, unique_notes, dropout=0.3, output_emb=100, rnn_unit=128, dense_unit=64):
    inputs = tf.keras.layers.Input(shape=(seq_len,))
    embedding = tf.keras.layers.Embedding(input_dim=unique_notes+1, output_dim=output_emb, input_length=seq_len)(inputs)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit, return_sequences=True))(embedding)
    forward_pass , att_vector = SeqSelfAttention(
        return_attention=True,
        attention_activation='sigmoid',
        attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
        attention_width=50,
        kernel_regularizer=tf.keras.regularizers.l2(1e-4),
        bias_regularizer=tf.keras.regularizers.l1(1e-4),
        attention_regularizer_weight=1e-4,
        )(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit, return_sequences=True))(forward_pass)
    forward_pass , att_vector2 = SeqSelfAttention(
        return_attention=True,
        attention_activation='sigmoid',
        attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
        attention_width=50,
        kernel_regularizer=tf.keras.regularizers.l2(1e-4),
        bias_regularizer=tf.keras.regularizers.l1(1e-4),
        attention_regularizer_weight=1e-4,
        )(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit))(forward_pass)
    forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
    forward_pass = tf.keras.layers.Dense(dense_unit)(forward_pass)
    forward_pass = tf.keras.layers.LeakyReLU()(forward_pass)
    outputs = tf.keras.layers.Dense(unique_notes+1, activation = "softmax")(forward_pass)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='generate_scores_rnn')
    return model

In [16]:
notes = [y for x in unique_target for y in x]

In [20]:
unique_notes = max(notes)

In [23]:
model = create_model(50, unique_notes+1)

In [24]:
model.summary()

Model: "generate_scores_rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 100)           10995600  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 256)           176640    
_________________________________________________________________
seq_self_attention (SeqSelfA [(None, 50, 256), (None,  65537     
_________________________________________________________________
dropout (Dropout)            (None, 50, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50, 256)           296448    
_________________________________________________________________
seq_self_attention_1 (SeqSel [(None, 50, 256), 

In [25]:
optimizer = Nadam()

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 model=model)
checkpoint_dir = 'models/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
loss_fn = sparse_categorical_crossentropy

In [231]:
import rnn_models
# from rnn_models import SeqSelfAttention
# from rnn_models import TrainModel

In [32]:
## Imports
import json
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.losses import sparse_categorical_crossentropy # used for integer targets
from tensorflow.keras.optimizers import Nadam
from tqdm import tqdm, tqdm_notebook
import numpy as np
import random
from random import shuffle, seed, sample


## Class for self sequecing layer from github
class SeqSelfAttention(tf.keras.layers.Layer):

    ATTENTION_TYPE_ADD = 'additive'
    ATTENTION_TYPE_MUL = 'multiplicative'

    def __init__(self,
                 units=32,
                 attention_width=None,
                 attention_type=ATTENTION_TYPE_ADD,
                 return_attention=False,
                 history_only=False,
                 kernel_initializer='glorot_normal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 use_additive_bias=True,
                 use_attention_bias=True,
                 attention_activation=None,
                 attention_regularizer_weight=0.0,
                 **kwargs):
        """Layer initialization.
        For additive attention, see: https://arxiv.org/pdf/1806.01264.pdf
        :param units: The dimension of the vectors that used to calculate the attention weights.
        :param attention_width: The width of local attention.
        :param attention_type: 'additive' or 'multiplicative'.
        :param return_attention: Whether to return the attention weights for visualization.
        :param history_only: Only use historical pieces of data.
        :param kernel_initializer: The initializer for weight matrices.
        :param bias_initializer: The initializer for biases.
        :param kernel_regularizer: The regularization for weight matrices.
        :param bias_regularizer: The regularization for biases.
        :param kernel_constraint: The constraint for weight matrices.
        :param bias_constraint: The constraint for biases.
        :param use_additive_bias: Whether to use bias while calculating the relevance of inputs features
                                  in additive mode.
        :param use_attention_bias: Whether to use bias while calculating the weights of attention.
        :param attention_activation: The activation used for calculating the weights of attention.
        :param attention_regularizer_weight: The weights of attention regularizer.
        :param kwargs: Parameters for parent class.
        """
        self.supports_masking = True
        self.units = units
        self.attention_width = attention_width
        self.attention_type = attention_type
        self.return_attention = return_attention
        self.history_only = history_only
        if history_only and attention_width is None:
            self.attention_width = int(1e9)

        self.use_additive_bias = use_additive_bias
        self.use_attention_bias = use_attention_bias
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.kernel_regularizer = tf.keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = tf.keras.regularizers.get(bias_regularizer)
        self.kernel_constraint = tf.keras.constraints.get(kernel_constraint)
        self.bias_constraint = tf.keras.constraints.get(bias_constraint)
        self.attention_activation = tf.keras.activations.get(attention_activation)
        self.attention_regularizer_weight = attention_regularizer_weight
        self._backend = tf.keras.backend.backend()

        if attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self.Wx, self.Wt, self.bh = None, None, None
            self.Wa, self.ba = None, None
        elif attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self.Wa, self.ba = None, None
        else:
            raise NotImplementedError('No implementation for attention type : ' + attention_type)

        super(SeqSelfAttention, self).__init__(**kwargs)

    def get_config(self):
        config = {
            'units': self.units,
            'attention_width': self.attention_width,
            'attention_type': self.attention_type,
            'return_attention': self.return_attention,
            'history_only': self.history_only,
            'use_additive_bias': self.use_additive_bias,
            'use_attention_bias': self.use_attention_bias,
            'kernel_initializer': tf.keras.regularizers.serialize(self.kernel_initializer),
            'bias_initializer': tf.keras.regularizers.serialize(self.bias_initializer),
            'kernel_regularizer': tf.keras.regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': tf.keras.regularizers.serialize(self.bias_regularizer),
            'kernel_constraint': tf.keras.constraints.serialize(self.kernel_constraint),
            'bias_constraint': tf.keras.constraints.serialize(self.bias_constraint),
            'attention_activation': tf.keras.activations.serialize(self.attention_activation),
            'attention_regularizer_weight': self.attention_regularizer_weight,
        }
        base_config = super(SeqSelfAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]
        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self._build_additive_attention(input_shape)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self._build_multiplicative_attention(input_shape)
        super(SeqSelfAttention, self).build(input_shape)

    def _build_additive_attention(self, input_shape):
        feature_dim = input_shape[2]

        self.Wt = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wt'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        self.Wx = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wx'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_additive_bias:
            self.bh = self.add_weight(shape=(self.units,),
                                      name='{}_Add_bh'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

        self.Wa = self.add_weight(shape=(self.units, 1),
                                  name='{}_Add_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Add_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

    def _build_multiplicative_attention(self, input_shape):
        feature_dim = input_shape[2]

        self.Wa = self.add_weight(shape=(feature_dim, feature_dim),
                                  name='{}_Mul_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Mul_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

    def call(self, inputs, mask=None, **kwargs):
        if isinstance(inputs, list):
            inputs, positions = inputs
            positions = K.cast(positions, 'int32')
            mask = mask[1]
        else:
            positions = None

        input_len = K.shape(inputs)[1]

        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            e = self._call_additive_emission(inputs)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            e = self._call_multiplicative_emission(inputs)

        if self.attention_activation is not None:
            e = self.attention_activation(e)
        e = K.exp(e - K.max(e, axis=-1, keepdims=True))
        if self.attention_width is not None:
            ones = tf.ones((input_len, input_len))
            if self.history_only:
                local = tf.linalg.band_part(
                    ones,
                    K.minimum(input_len, self.attention_width - 1),
                    0,
                )
            else:
                local = tf.linalg.band_part(
                    ones,
                    K.minimum(input_len, self.attention_width // 2),
                    K.minimum(input_len, (self.attention_width - 1) // 2),
                )
            e = e * K.expand_dims(local, 0)
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            e = K.permute_dimensions(K.permute_dimensions(e * mask, (0, 2, 1)) * mask, (0, 2, 1))

        # a_{t} = \text{softmax}(e_t)
        s = K.sum(e, axis=-1)
        s = K.tile(K.expand_dims(s, axis=-1), K.stack([1, 1, input_len]))
        a = e / (s + K.epsilon())

        # l_t = \sum_{t'} a_{t, t'} x_{t'}
        v = K.batch_dot(a, inputs)
        if self.attention_regularizer_weight > 0.0:
            self.add_loss(self._attention_regularizer(a))

        if positions is not None:
            pos_num = K.shape(positions)[1]
            batch_indices = K.tile(K.expand_dims(K.arange(K.shape(inputs)[0]), axis=-1), K.stack([1, pos_num]))
            pos_indices = K.stack([batch_indices, positions], axis=-1)
            v = tf.gather_nd(v, pos_indices)
            a = tf.gather_nd(a, pos_indices)

        if self.return_attention:
            return [v, a]
        return v

    def _call_additive_emission(self, inputs):
        input_shape = K.shape(inputs)
        batch_size, input_len = input_shape[0], input_shape[1]

        # h_{t, t'} = \tanh(x_t^T W_t + x_{t'}^T W_x + b_h)
        q, k = K.dot(inputs, self.Wt), K.dot(inputs, self.Wx)
        q = K.tile(K.expand_dims(q, 2), K.stack([1, 1, input_len, 1]))
        k = K.tile(K.expand_dims(k, 1), K.stack([1, input_len, 1, 1]))
        if self.use_additive_bias:
            h = K.tanh(q + k + self.bh)
        else:
            h = K.tanh(q + k)

        # e_{t, t'} = W_a h_{t, t'} + b_a
        if self.use_attention_bias:
            e = K.reshape(K.dot(h, self.Wa) + self.ba, (batch_size, input_len, input_len))
        else:
            e = K.reshape(K.dot(h, self.Wa), (batch_size, input_len, input_len))
        return e

    def _call_multiplicative_emission(self, inputs):
        # e_{t, t'} = x_t^T W_a x_{t'} + b_a
        e = K.batch_dot(K.dot(inputs, self.Wa), K.permute_dimensions(inputs, (0, 2, 1)))
        if self.use_attention_bias:
            e = e + self.ba
        return e

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape, pos_shape = input_shape
            output_shape = (input_shape[0], pos_shape[1], input_shape[2])
        else:
            output_shape = input_shape
        if self.return_attention:
            attention_shape = (input_shape[0], output_shape[1], input_shape[1])
            return [output_shape, attention_shape]
        return output_shape

    def compute_mask(self, inputs, mask=None):
        if isinstance(inputs, list):
            mask = mask[1]
        if self.return_attention:
            return [mask, None]
        return mask

    def _attention_regularizer(self, attention):
        batch_size = K.cast(K.shape(attention)[0], K.floatx())
        input_len = K.shape(attention)[-1]
        return self.attention_regularizer_weight * K.sum(K.square(K.batch_dot(
            attention,
            K.permute_dimensions(attention, (0, 2, 1))) - tf.eye(input_len))) / batch_size

    @staticmethod
    def get_custom_objects():
        return {'SeqSelfAttention': SeqSelfAttention}

# Training Model Class
class TrainModel:

    def __init__(self, epochs, sample_train, sample_target, batch_nnet_size, batch_song, optimizer, checkpoint, loss_fn,checkpoint_prefix, total_songs, model):
        self.epochs = epochs
        self.sample_train = sample_train
        self.sample_target = sample_target
        self.batch_nnet_size = batch_nnet_size
        self.batch_song = batch_song
        self.optimizer = optimizer
        self.checkpoint = checkpoint
        self.loss_fn = loss_fn
        self.checkpoint_prefix = checkpoint_prefix
        self.total_songs = total_songs
        self.model = model

    def train(self):
        for epoch in tqdm_notebook(range(self.epochs),desc='epochs'):
            # for each epoch, shuffle the list of all the songs
            c = list(zip(self.sample_train, self.sample_target))
            shuffle(c)
            self.sample_train, self.sample_target = zip(*c)
            loss_total = 0
            steps = 0
            steps_nnet = 0
            # Iterate all songs by the length of sample input (total_songs) and batches (batch_song)
            for i in tqdm_notebook(range(0,self.total_songs, self.batch_song), desc='MUSIC'):
                # EXAMPLE: [0,5,10,15,20] FOR TOTAL_SONGS = 20 AND BATCH_SONG = 5
                steps += 1
                #inputs_nnet_large, outputs_nnet_large = generate_batch_song(
                 #   self.sample_input, self.batch_song, start_index=i, fs=self.frame_per_second,
                  #  seq_len=seq_len, use_tqdm=False) # We use the function that have been defined here
                #inputs_nnet_large = np.array(self.note_tokenizer.transform(inputs_nnet_large), dtype=np.int32)
                #outputs_nnet_large = np.array(self.note_tokenizer.transform(outputs_nnet_large), dtype=np.int32)

                # EXAMPLE LARGE INPUTS = ARRAY([1,2,3,4],[2,3,4,5],[2,3,4,5],[2,3,4,5],[1,2,3,4])
                input_batch = [y for x in self.sample_train[i:i+self.batch_song] for y in x]
                output_batch = [y for x in self.sample_target[i:i+self.batch_song] for y in x]
                c = list(zip(input_batch, output_batch))
                sample_in = sample(c, 10000)
                input_batch, output_batch = zip(*sample_in)
                inputs_nnet_large = np.array(input_batch)
                outputs_nnet_large = np.array(output_batch)

                # Get an index of all windows in a song
                index_shuffled = np.arange(start=0, stop=len(inputs_nnet_large))
                np.random.shuffle(index_shuffled)

                for nnet_steps in tqdm_notebook(range(0,len(index_shuffled),self.batch_nnet_size)):
                    steps_nnet += 1
                    current_index = index_shuffled[nnet_steps:nnet_steps+self.batch_nnet_size]

                    inputs_nnet, outputs_nnet = inputs_nnet_large[current_index], outputs_nnet_large[current_index]

                    # To make sure no exception thrown by tensorflow on autograph
                    if len(inputs_nnet) // self.batch_nnet_size != 1:
                        break
                    loss = self.train_step(inputs_nnet, outputs_nnet)
                    loss_total += tf.math.reduce_sum(loss)
                    if steps_nnet % 20 == 0:
                        print("epochs {} | Steps {} | total loss : {}".format(epoch + 1, steps_nnet,loss_total))

                    #checkpoint.save(file_prefix = self.checkpoint_prefix)

    @tf.function
    def train_step(self, inputs, targets):
        with tf.GradientTape() as tape:
            prediction = self.model(inputs)
            loss = self.loss_fn(targets, prediction)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss


In [26]:
seq_len = 50
EPOCHS = 2
BATCH_SONG = 10
BATCH_NNET_SIZE = 96
TOTAL_SONGS = len(unique_target)
FRAME_PER_SECOND = 5

In [29]:
train_class = TrainModel(EPOCHS, unique_train, unique_target, BATCH_NNET_SIZE, BATCH_SONG, optimizer, checkpoint, loss_fn, checkpoint_prefix, 
                        TOTAL_SONGS, model)
train_class.train()

epochs 1 | Steps 20 | total loss : 15204.35546875
epochs 1 | Steps 40 | total loss : 30480.626953125
epochs 1 | Steps 60 | total loss : 45556.1171875
epochs 1 | Steps 80 | total loss : 60398.91796875
epochs 1 | Steps 100 | total loss : 75496.8046875


epochs 1 | Steps 120 | total loss : 89402.5859375
epochs 1 | Steps 140 | total loss : 103897.75
epochs 1 | Steps 160 | total loss : 118101.7421875
epochs 1 | Steps 180 | total loss : 132250.125
epochs 1 | Steps 200 | total loss : 146228.03125


epochs 1 | Steps 220 | total loss : 159385.015625
epochs 1 | Steps 240 | total loss : 173710.984375
epochs 1 | Steps 260 | total loss : 188204.078125
epochs 1 | Steps 280 | total loss : 202152.609375
epochs 1 | Steps 300 | total loss : 216107.09375


epochs 1 | Steps 320 | total loss : 229489.34375
epochs 1 | Steps 340 | total loss : 243486.703125
epochs 1 | Steps 360 | total loss : 257412.921875
epochs 1 | Steps 380 | total loss : 270663.21875
epochs 1 | Steps 400 | total loss : 283806.90625


epochs 1 | Steps 440 | total loss : 310686.28125
epochs 1 | Steps 460 | total loss : 324775.375
epochs 1 | Steps 480 | total loss : 338483.9375
epochs 1 | Steps 500 | total loss : 352154.1875
epochs 1 | Steps 520 | total loss : 365644.34375


...

epochs 2 | Steps 640 | total loss : 381170.9375
epochs 2 | Steps 660 | total loss : 393098.71875
epochs 2 | Steps 680 | total loss : 405084.40625
epochs 2 | Steps 700 | total loss : 416521.6875
epochs 2 | Steps 720 | total loss : 428324.625


epochs 2 | Steps 740 | total loss : 439123.75
epochs 2 | Steps 760 | total loss : 450980.125
epochs 2 | Steps 780 | total loss : 462519.71875
epochs 2 | Steps 800 | total loss : 473749.03125
epochs 2 | Steps 820 | total loss : 485104.46875


epochs 2 | Steps 860 | total loss : 507611.75
epochs 2 | Steps 880 | total loss : 519662.8125
epochs 2 | Steps 900 | total loss : 531471.9375
epochs 2 | Steps 920 | total loss : 543131.625
epochs 2 | Steps 940 | total loss : 554746.3125


epochs 2 | Steps 960 | total loss : 565464.125
epochs 2 | Steps 980 | total loss : 576777.5
epochs 2 | Steps 1000 | total loss : 587747.125
epochs 2 | Steps 1020 | total loss : 598834.9375
epochs 2 | Steps 1040 | total loss : 609534.5


epochs 2 | Steps 1060 | total loss : 620000.25
epochs 2 | Steps 1080 | total loss : 631571.3125
epochs 2 | Steps 1100 | total loss : 642735.5625
epochs 2 | Steps 1120 | total loss : 653739.9375
epochs 2 | Steps 1140 | total loss : 664876.5




In [41]:
saver = tf.train.Saver()
saver.save(model, 'chopin_s107_ep2') 

AttributeError: module 'tensorflow_core._api.v2.train' has no attribute 'Saver'

In [42]:
model.save_weights('chopin_weights')

In [43]:
model.save('chopin_model.h5')

TypeError: super(type, obj): obj must be an instance or subtype of type

In [44]:
# serialize model to JSON
model_json = model.to_json()
with open("chopin_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("chopin_model.h5")
print("Saved model to disk")

TypeError: super(type, obj): obj must be an instance or subtype of type